In [58]:
import nmap3,json

#get api key from apikey.txt file
def getKey():
    try:
        file = open("apikey.txt")
        key = file.read()
        file.close()
        return key
    except Exception as err:
        print("Error with getting API key: ", err)
        return ""
    
#version scan
def versionScan():
    nmap = nmap3.Nmap()
    target = "scanme.nmap.org"
    result = nmap.nmap_version_detection(target)
    return result
    
    
def parseVersion(result):
    parsedResult = {}
    entries = []
    for key in result:
        if '.' in key:
            host = key
            for subkey in result[key]["ports"]:
                if "cpe" in subkey:
                    for entry in subkey["cpe"]:
                        if not entry["cpe"] == "cpe:/o:linux:linux_kernel":
                            entries.append(entry["cpe"].replace("/","2.3:"))
                    parsedResult[host] = entries
    return parsedResult

In [60]:
import nvdlib

result = versionScan()
result = parseVersion(result)
print(result)
key = getKey()
for host in result:
    print("CVEs of ", host, ": ")
    for cpe in result[host]:
        print(cpe)
        r = nvdlib.searchCVE(cpeName=cpe,key=key,limit=5)
        for item in r:
            print(item)

{'45.33.32.156': ['cpe:2.3:a:openbsd:openssh:6.6.1p1', 'cpe:2.3:a:apache:http_server:2.4.7']}
CVEs of  45.33.32.156 : 
cpe:2.3:a:openbsd:openssh:6.6.1p1
{'id': 'CVE-2007-2768', 'sourceIdentifier': 'cve@mitre.org', 'published': '2007-05-21T20:30:00.000', 'lastModified': '2021-04-01T15:32:46.563', 'vulnStatus': 'Analyzed', 'descriptions': [{'lang': 'en', 'value': 'OpenSSH, when using OPIE (One-Time Passwords in Everything) for PAM, allows remote attackers to determine the existence of certain user accounts, which displays a different response if the user account exists and is configured to use one-time passwords (OTP), a similar issue to CVE-2007-2243.'}, {'lang': 'es', 'value': 'OpenSSH, cuando utiliza OPIE(One-Time Passwords in Everything) para PAM, permiet a atacantes remotos determinar la existencia de ciertas cuentas de usuarios, lo cual muestra una respuesta diferente si la cuenta de usuario existe y si está configurada para utilizar one-time passwords (OTP), un asunto similar es e